## Import thư viện 

In [ ]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Load dữ liệu

In [ ]:
# Chọn phiên bản mới nhất và ngôn ngữ tiếng Việt
url = 'https://ddragon.leagueoflegends.com/api/versions.json'
r = requests.get(url)
version = r.json()[0]
language = "vi_VN"

In [ ]:
# Tạo URL 
url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/{language}/item.json"

# Gửi yêu cầu GET đến API của Riot Games
response = requests.get(url)

# Kiểm tra mã trạng thái phản hồi có phải là 200 hay không
if response.status_code == 200:
    # Lấy thông tin vật phẩm 
    items = response.json()['data']
    print('Lấy thông tin vật phẩm thành công')
else:
    # Xử lý lỗi tại đây nếu có
    print(f"Request failed with error code {response.status_code}")

In [ ]:
# Tạo URL 
url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/{language}/map.json"
# Gửi yêu cầu GET đến API của Riot Games
response = requests.get(url)

# Kiểm tra mã trạng thái phản hồi có phải là 200 hay không
if response.status_code == 200:
    # Lấy thông tin bản đồ
    maps = response.json()['data']
    # Đưa maps sang dataframe
    maps = pd.DataFrame(maps).T
    print('Lấy thông tin bản đồ thành công')
else:
    # Xử lý lỗi tại đây nếu có
    print(f"Request failed with error code {response.status_code}")

## Tiền xử lý dữ liệu

In [ ]:
# Tạo DataFrame từ thông tin vật phẩm
df = pd.DataFrame(items).T
# Lấy tên, thông số và giá của vật phẩm
item = df[['name', 'stats', 'gold', 'maps']]

In [ ]:
# Xóa các giá trị False trong dict maps
item['maps'] = item['maps'].apply(lambda x: {k: v for k, v in x.items() if v != False})

In [ ]:
# Tách cột stats, gold thành các cột riêng biệt
item = pd.concat([item.drop(['stats'], axis=1), item['stats'].apply(pd.Series)], axis=1)
item = pd.concat([item.drop(['gold'], axis=1), item['gold'].apply(pd.Series)], axis=1)


In [ ]:
# Tên cột 
item.columns

In [ ]:
# Đổi tên cột
item = item.rename(columns={
'name' : 'Tên vật phẩm',
'maps' : 'Bản đồ',
'FlatMovementSpeedMod' : 'Tốc độ di chuyển',
'FlatHPPoolMod' : 'Máu',
'FlatCritChanceMod' : 'Tỉ lệ chí mạng',
'FlatMagicDamageMod' : 'Sát thương phép',
'FlatMPPoolMod' : 'Năng lượng',
'FlatArmorMod' : 'Giáp',
'FlatSpellBlockMod' : 'Kháng phép',
'FlatPhysicalDamageMod' : 'Sát thương vật lý',
'PercentAttackSpeedMod' : 'Tốc độ đánh',
'PercentLifeStealMod' : 'Hút máu',
'FlatHPRegenMod' : 'Hồi máu',
'PercentMovementSpeedMod' : 'Tỉ lệ tốc độ di chuyển',
'base' : 'Giá nâng cấp',
'purchasable' : 'Có thể mua',
'total' : 'Giá mua',
'sell' : 'Giá bán'
})


In [ ]:
# Điền giá trị NaN bằng 0
item = item.fillna(0)

## Tính giá trị vật phẩm dựa vào chỉ số

In [ ]:
ChiSo = ['Tốc độ di chuyển', 'Máu', 'Tỉ lệ chí mạng',
       'Sát thương phép', 'Năng lượng', 'Giáp', 'Kháng phép',
       'Sát thương vật lý', 'Tốc độ đánh', 'Hút máu', 'Hồi máu',
       'Tỉ lệ tốc độ di chuyển']

In [ ]:
# Tính số tiền mỗi 1 đơn vị chỉ số
price = pd.DataFrame()

for col in ChiSo:
    price[col] = item['Giá mua'] / item[col]     

In [ ]:
# Lưu lại tên vật phẩm và giá trị chỉ số cơ sở
min_price = pd.DataFrame()
for col in ChiSo:
    min_price[col] = item.loc[price[col].idxmin()]

In [ ]:
# Tính giá trị của vật phẩm
item['Giá trị chỉ số'] = 0
for col in item.columns:
    if col in ChiSo:
        item['Giá trị chỉ số'] += item[col] * price[col]

In [ ]:
# Thêm cột hiệu quả 
item['Hiệu quả'] = item.apply(lambda row: row['Giá trị chỉ số'] / row['Giá mua'] if row['Giá mua'] != 0 else 0, axis=1)

## Lưu lại kết quả để trực quan hóa


In [ ]:
#item drop Có thể mua column 
item = item.drop(['Có thể mua'], axis=1)

# Tách maps True thành các hàng dọc 
item = item.explode('Bản đồ')
# Đổi tên item['maps'] dựa vào thông tin bản đồ
item['Bản đồ'] = item['Bản đồ'].replace(maps['MapName'])

In [ ]:
item.to_csv('item.csv', index=False, encoding='utf-8-sig')
min_price.to_csv('min_price.csv', index=False, encoding='utf-8-sig')